In [21]:
import chromadb

client = chromadb.Client()

collection = client.get_or_create_collection(name="my_collection")


In [22]:
from langchain_community.document_loaders import NotebookLoader



documents = NotebookLoader("3-cartpole.ipynb").load()
documents

[Document(metadata={'source': '3-cartpole.ipynb'}, page_content='\'markdown\' cell: \'[\'# cartpole\']\'\n\n\'code\' cell: \'[\'import matplotlib.pyplot as plt\\n\', \'import gymnasium as gym\\n\', \'from IPython import display\\n\', \'%matplotlib inline\\n\', \'\\n\']\'\n\n\'code\' cell: \'[\'env = gym.make("CartPole-v1", render_mode="rgb_array")\\n\', \'observation, info = env.reset()\\n\', \'\\n\', \'terminated = False\\n\', \'truncated = False\\n\', \'\\n\', \'while not terminated and not truncated:\\n\', \'    new_state, reward, terminated, truncated, _ = env.step(env.action_space.sample())\\n\', \'    observation = new_state\\n\', \'env.close()\']\'\n\n\'code\' cell: \'[\'\\n\', \'## ploting on jupyter\\n\', \'plt.figure(figsize=(2, 1))\\n\', \'for i in range(25):\\n\', \'   plt.imshow(env.render())\\n\', \'   display.display(plt.gcf())\\n\', \'   display.clear_output(wait=True)\\n\', \'   env.step(env.action_space.sample()) # take a random action\\n\', \'\\n\', \'env.close()\\n\

In [23]:

from langchain_text_splitters import PythonCodeTextSplitter

text_splitter = PythonCodeTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = text_splitter.split_documents(documents)
chunks

[Document(metadata={'source': '3-cartpole.ipynb'}, page_content='\'markdown\' cell: \'[\'# cartpole\']\'\n\n\'code\' cell: \'[\'import matplotlib.pyplot as plt\\n\', \'import gymnasium as gym\\n\', \'from IPython import display\\n\', \'%matplotlib inline\\n\', \'\\n\']\'\n\n\'code\' cell: \'[\'env = gym.make("CartPole-v1", render_mode="rgb_array")\\n\', \'observation, info = env.reset()\\n\', \'\\n\', \'terminated = False\\n\', \'truncated = False\\n\', \'\\n\', \'while not terminated and not truncated:\\n\', \'    new_state, reward, terminated, truncated, _ = env.step(env.action_space.sample())\\n\', \'    observation = new_state\\n\', \'env.close()\']\'\n\n\'code\' cell: \'[\'\\n\', \'## ploting on jupyter\\n\', \'plt.figure(figsize=(2, 1))\\n\', \'for i in range(25):\\n\', \'   plt.imshow(env.render())\\n\', \'   display.display(plt.gcf())\\n\', \'   display.clear_output(wait=True)\\n\', \'   env.step(env.action_space.sample()) # take a random action\\n\', \'\\n\', \'env.close()\\n\

In [24]:
documents = []
metadata = []
ids = []

for i, chunk in enumerate(chunks):
    documents.append(chunk.page_content)
    metadata.append(chunk.metadata)
    ids.append(f"chunk_{i}")

documents

['\'markdown\' cell: \'[\'# cartpole\']\'\n\n\'code\' cell: \'[\'import matplotlib.pyplot as plt\\n\', \'import gymnasium as gym\\n\', \'from IPython import display\\n\', \'%matplotlib inline\\n\', \'\\n\']\'\n\n\'code\' cell: \'[\'env = gym.make("CartPole-v1", render_mode="rgb_array")\\n\', \'observation, info = env.reset()\\n\', \'\\n\', \'terminated = False\\n\', \'truncated = False\\n\', \'\\n\', \'while not terminated and not truncated:\\n\', \'    new_state, reward, terminated, truncated, _ = env.step(env.action_space.sample())\\n\', \'    observation = new_state\\n\', \'env.close()\']\'\n\n\'code\' cell: \'[\'\\n\', \'## ploting on jupyter\\n\', \'plt.figure(figsize=(2, 1))\\n\', \'for i in range(25):\\n\', \'   plt.imshow(env.render())\\n\', \'   display.display(plt.gcf())\\n\', \'   display.clear_output(wait=True)\\n\', \'   env.step(env.action_space.sample()) # take a random action\\n\', \'\\n\', \'env.close()\\n\']\'',
 '\'code\' cell: \'[\'import numpy as np\\n\', \'env = g

In [27]:
collection.upsert(
    documents=documents,
    metadatas=metadata,
    ids=ids
)
collection.count()

2

In [28]:
import openai
import requests
import json
from typing import List, Dict, Any

# =============================================================================
# SETUP AND CONFIGURATION
# =============================================================================

# Ollama typically runs on localhost:11434
OLLAMA_BASE_URL = "http://localhost:11434"
OLLAMA_API_URL = f"{OLLAMA_BASE_URL}/v1"

# Configure OpenAI client to use Ollama
openai_client = openai.OpenAI(
    base_url=OLLAMA_API_URL,
    api_key="ollama"  # Ollama doesn't require a real API key, but OpenAI client expects one
)

system_prompt = """
You are a helpful assistant. You answer questions about the Python script provided.
but you only answer based on the content of the script. You don't use any prior knowledge and you don't make up answers.
If you don't know the answer, just say "I don't know".

the data:
"""


In [29]:
user_query = "is the reward variable being used in the code?"

results = collection.query(
    query_texts=[user_query],
    n_results=1
)
results

{'ids': [['chunk_1']],
 'embeddings': None,
 'documents': [['\'code\' cell: \'[\'import numpy as np\\n\', \'env = gym.make("CartPole-v1", render_mode="rgb_array")\\n\', \'observation, info = env.reset()\\n\', \'\\n\', \'q = np.zeros((env.observation_space.n, env.action_space.n))\\n\', \'\\n\', \'terminated = False\\n\', \'truncated = False\\n\', \'\\n\', \'learning_rate_a = 0.9\\n\', \'discount_factor_g = 0.9\\n\', \'\\n\', \'\\n\', \'while not terminated and not truncated:\\n\', \'    new_state, reward, terminated, truncated, _ = env.step(env.action_space.sample())\\n\', \'    observation = new_state\\n\', \'\\n\', \'env.close()\']\'\n\n\'code\' cell: \'[]\'']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'source': '3-cartpole.ipynb'}]],
 'distances': [[1.2238738536834717]]}

In [32]:
response = openai_client.chat.completions.create(
    model="llama3.2",
    messages=[
        {"role": "system", "content": system_prompt + str(results['documents'])},
        {"role": "user", "content": user_query}
    ]
)
response.choices[0].message.content

"The `reward` variable is not explicitly used in the provided code. It seems that it's being stored as part of the new state (`new_state`) for later use, but its value isn't being utilized anywhere within the loop."

## Summary

In this notebook, we read `3-cartpole.ipynb` file and store it on chromadb Vector Database.
The Vector database then divide the file on small chunks and allow us to seach and retrieve the most relevant chunck based on our query

Later, we ask the question `"is the reward variable being used in the code?"` so we:
1. search the vector database and retrieve the most relevant chunk 
2. create a custom system prompt adding instructions + the chunck data
3. send to openai api the system prompt + the user question 

In conclusion, the agent give us a not 100% correct answer that "The `reward` variable is not explicitly used in the provided code. It seems that it's being stored as part of the new state (`new_state`) for later use"
where the first part is correct but the `It seems that it's being stored as part of the new state (new_state) for later use` part is false.

Here we are using a very small model running on local environment which may have caused the imprecise answer, also, I could improve the vector store parameters which I didn't paid much attention yet.